**Run the following two cells before you begin.**

In [1]:
%autosave 10

Autosaving every 10 seconds


In [61]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [62]:
# Import the data set
df=pd.read_csv('cleaned_data.csv')
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


In [63]:
# Define the sigmoid function
def sigmoid(x):
    return 1/(1+np.exp(-x))

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [64]:
X=df.loc[:,['PAY_1','LIMIT_BAL']]
Y=df.loc[:,['default payment next month']]
X.head()

,PAY_1,LIMIT_BAL
0,2,20000
1,-1,120000
2,0,90000
3,0,50000
4,-1,50000


In [65]:
# Create a train/test split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=24)
X_train.head()

,PAY_1,LIMIT_BAL
6409,0,120000
4663,2,190000
13763,-1,420000
7542,2,30000
14518,0,120000


______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [66]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [67]:
# Fit the logistic regression model on training data
clf.fit(X_train,Y_train)

C:\Users\Ishan SS\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [68]:
# Make predictions using `.predict()`
Y_pred=clf.predict(X_test)

In [69]:
# Find class probabilities using `.predict_proba()`
Y_pred_prob=clf.predict_proba(X_test)

In [70]:
Y_pred_prob

array([[0.74826924, 0.25173076],
       [0.584297  , 0.415703  ],
       [0.79604453, 0.20395547],
       ...,
       [0.584297  , 0.415703  ],
       [0.82721498, 0.17278502],
       [0.66393435, 0.33606565]])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [71]:
X.shape

(26664, 2)

In [72]:
# Add column of 1s to features
one=np.ones((X_test.shape[0],1),dtype=int)
X_test=np.append(X_test,one,axis=1)
print(X_test.shape)
X_test

(5333, 3)


array([[     2, 160000,      1],
       [     1,  50000,      1],
       [    -1, 200000,      1],
       ...,
       [    -1,  50000,      1],
       [     1, 230000,      1],
       [     2, 100000,      1]], dtype=int64)

In [73]:
# Get coefficients and intercepts from trained model
coeffs=clf.coef_
intercept=clf.intercept_
coeffs,intercept

(array([[ 8.27451187e-11, -6.80876727e-06]]), array([-6.57647457e-11]))

In [74]:
coeffs=np.append(coeffs,intercept)
coeffs

array([ 8.27451187e-11, -6.80876727e-06, -6.57647457e-11])

In [75]:
Z=np.dot(coeffs,X_test.T)
print(len(Z))
Z

5333


array([-1.08940276, -0.34043836, -1.36175345, ..., -0.34043836,
       -1.56601647, -0.68087673])

In [76]:
# Manually calculate predicted probabilities
pred_proba=sigmoid(Z)
pred_proba

array([0.25173076, 0.415703  , 0.20395547, ..., 0.415703  , 0.17278502,
       0.33606565])

In [77]:
pred_prob=[]
for i in range(len(pred_proba)):
    temp=[]
    temp.append(1-pred_proba[i])
    temp.append(pred_proba[i])
    pred_prob.append(temp)
pred_prob

[[0.7482692412207628, 0.2517307587792373],
 [0.584297002694049, 0.4157029973059509],
 [0.7960445320597713, 0.20395546794022873],
 [0.7960445320060341, 0.20395546799396586],
 [0.584297002694049, 0.4157029973059509],
 [0.9206459615683965, 0.07935403843160352],
 [0.9206459615683965, 0.07935403843160352],
 [0.663934345627771, 0.336065654372229],
 [0.7352286839145739, 0.26477131608542614],
 [0.7608764148571872, 0.23912358514281284],
 [0.6329053995362448, 0.36709460046375514],
 [0.5676699048051779, 0.43233009519482213],
 [0.796044532046337, 0.20395546795366298],
 [0.5339913254281781, 0.466008674571822],
 [0.7960445319522969, 0.2039554680477031],
 [0.8852008186486576, 0.11479918135134237],
 [0.663934345627771, 0.336065654372229],
 [0.6329053995554696, 0.3670946004445304],
 [0.8458207820248429, 0.15417921797515716],
 [0.600734397342152, 0.399265602657848],
 [0.6329053994977956, 0.36709460050220444],
 [0.9300417968069964, 0.0699582031930036],
 [0.8272149830344946, 0.17278501696550547],
 [0.5842

In [86]:
pred_prob=np.array(pred_prob)

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [80]:
# Manually calculate predicted classes
classes=[]
for i in range(len(pred_prob)):
    if pred_prob[i][0]>0.5:
        classes.append(0.0)
    else:
        classes.append(1.0)
classes

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [ ]:
# Compare to scikit-learn's predicted classes


______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [22]:
from sklearn.metrics import roc_curve,roc_auc_score

In [24]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
ruc_score_skl=roc_auc_score(Y_test,Y_pred_prob[:,1])
ruc_score_skl

0.627207450280691

In [87]:
pred_prob[0],Y_pred_prob[0]

(array([0.74826924, 0.25173076]), array([0.74826924, 0.25173076]))

In [88]:
# Use manually calculated predicted probabilities to calculate ROC AUC
ruc_score_man=roc_auc_score(Y_test,pred_prob[:,1])
ruc_score_man

0.627207450280691